In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
import keras
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Input, Conv2DTranspose, Concatenate, BatchNormalization, UpSampling2D
from keras.layers import  Dropout, Activation
from keras.optimizers import Adam, SGD
from keras.layers.advanced_activations import LeakyReLU
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras import backend as K
from keras.utils import plot_model
import tensorflow as tf
import glob
import random
import cv2
from random import shuffle
from osgeo import gdal

from pyproj import Proj, transform
from tqdm import tqdm
from shapely.geometry import Polygon

ModuleNotFoundError: No module named 'keras'

In [3]:
import os
import numpy as np
import cv2
from glob import glob
import tensorflow as tf
from sklearn.model_selection import train_test_split

path = "E:\\Citra\\UNet 128\\unet2122"
images = sorted(glob(os.path.join(path, "train/*")))
masks = sorted(glob(os.path.join(path, "train_masks/*")))

minx = -70
maxx = 70

a = images[1]
b = gdal.Open(a).ReadAsArray()
c = np.rollaxis(b, 0, 3)
d = (c - minx) / (maxx - minx)

e = masks[1]
f = gdal.Open(e).ReadAsArray()
g = np.expand_dims(f, axis=-1)
g.shape

#h = (g - minx) / (maxx - minx)


ModuleNotFoundError: No module named 'cv2'

In [ ]:

def load_data(path, split=0.1):
    images = sorted(glob(os.path.join(path, "train/*")))
    masks = sorted(glob(os.path.join(path, "train_masks/*")))

    total_size = len(images)
    valid_size = int(split * total_size)
    test_size = int(split * total_size)

    train_x, valid_x = train_test_split(images, test_size=valid_size, random_state=42)
    train_y, valid_y = train_test_split(masks, test_size=valid_size, random_state=42)

    train_x, test_x = train_test_split(train_x, test_size=test_size, random_state=42)
    train_y, test_y = train_test_split(train_y, test_size=test_size, random_state=42)

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)

In [ ]:
def read_image(path):
    path = path.decode()
    x = gdal.Open(path).ReadAsArray()
    x = np.rollaxis(x, 0, 3)
    x = (x - minx) / (maxx - minx)
    x = x.astype(np.float32)
    return x

In [ ]:
def read_mask(path):
    path = path.decode()
    x = gdal.Open(path).ReadAsArray()
    x[x == 2] = 1
    #x = np.rollaxis(x, 0, 3)
    x = np.expand_dims(x, axis=-1)
    x = x.astype(np.float32)
    return x

In [ ]:
def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([128, 128, 3])
    y.set_shape([128, 128, 1])
    return x, y

In [ ]:
def tf_dataset(x, y, batch=32):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.repeat()
    return dataset

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

def conv_block(x, num_filters):
    x = Conv2D(num_filters, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def build_model():
    size = 128
    num_filters = [16, 32, 48, 64]
    inputs = Input((size, size, 3))

    skip_x = []
    x = inputs

    ## Encoder
    for f in num_filters:
        x = conv_block(x, f)
        skip_x.append(x)
        x = MaxPool2D((2, 2))(x)

    ## Bridge
    x = conv_block(x, num_filters[-1])

    num_filters.reverse()
    skip_x.reverse()

    ## Decoder
    for i, f in enumerate(num_filters):
        x = UpSampling2D((2, 2))(x)
        xs = skip_x[i]
        x = Concatenate()([x, xs])
        x = conv_block(x, f)

    ## Output
    x = Conv2D(1, (1, 1), padding="same")(x)
    x = Activation("sigmoid")(x)

    return Model(inputs, x)


In [ ]:
import os
import numpy as np
import cv2
from glob import glob
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger, TensorBoard

In [ ]:
def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

In [ ]:
if __name__ == "__main__":
    ## Dataset
    path = "E:\\Citra\\UNet 128\\unet2122"
    (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(path)

In [ ]:
batch = 32
epochs = 1

In [ ]:
train_dataset = tf_dataset(train_x, train_y, batch=batch)
valid_dataset = tf_dataset(valid_x, valid_y, batch=batch)
print(train_dataset)
print(valid_dataset)


<RepeatDataset shapes: ((None, 128, 128, 3), (None, 128, 128, 1)), types: (tf.float32, tf.float32)>
<RepeatDataset shapes: ((None, 128, 128, 3), (None, 128, 128, 1)), types: (tf.float32, tf.float32)>


In [ ]:
model = build_model()
opt = tf.keras.optimizers.Adam()
metrics = ["accuracy", tf.keras.metrics.Recall(), tf.keras.metrics.Precision(), iou]
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=metrics)
model.summary()
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv2d_19 (Conv2D)              (None, 128, 128, 16) 448         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_18 (BatchNo (None, 128, 128, 16) 64          conv2d_19[0][0]                  
__________________________________________________________________________________________________
activation_19 (Activation)      (None, 128, 128, 16) 0           batch_normalization_18[0][0]     
_______________________________________________________________________________________

In [ ]:
callbacks = [
    ModelCheckpoint("files/model.h5"),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4),
    CSVLogger("files/data_2.csv"),
    TensorBoard(),
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=False)
]

In [ ]:
train_steps = len(train_x)//batch
valid_steps = len(valid_x)//batch

if len(train_x) % batch != 0:
    train_steps += 1
if len(valid_x) % batch != 0:
    valid_steps += 1

In [ ]:
model.fit(train_dataset,
          validation_data=valid_dataset,
          epochs=epochs,
          steps_per_epoch=train_steps,
          validation_steps=valid_steps,
          callbacks=callbacks)

Epoch 1/100
257/257 [==============================] - 29s 114ms/step - loss: 0.1533 - accuracy: 0.9691 - recall: 0.7624 - precision: 0.7999 - iou: 0.2983 - val_loss: 0.2517 - val_accuracy: 0.9346 - val_recall: 0.0755 - val_precision: 0.9964 - val_iou: 0.0806
Epoch 2/100
257/257 [==============================] - 26s 102ms/step - loss: 0.0986 - accuracy: 0.9716 - recall: 0.7791 - precision: 0.8191 - iou: 0.4210 - val_loss: 0.2279 - val_accuracy: 0.9340 - val_recall: 0.0669 - val_precision: 1.0000 - val_iou: 0.0912
Epoch 3/100
257/257 [==============================] - 27s 103ms/step - loss: 0.0790 - accuracy: 0.9744 - recall: 0.8011 - precision: 0.8372 - iou: 0.4919 - val_loss: 0.1728 - val_accuracy: 0.9472 - val_recall: 0.2566 - val_precision: 0.9895 - val_iou: 0.2096
Epoch 4/100
257/257 [==============================] - 26s 101ms/step - loss: 0.0723 - accuracy: 0.9755 - recall: 0.8083 - precision: 0.8455 - iou: 0.5223 - val_loss: 0.3512 - val_accuracy: 0.9293 - val_recall: 2.7943e-0

In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope
from tqdm import tqdm

In [ ]:
def mask_parse(mask):
    mask = np.squeeze(mask)
    mask = [mask, mask, mask]
    mask = np.transpose(mask, (1, 2, 0))
    return mask

In [ ]:
model.save("unet_flood2122.h5")

In [ ]:
batch_size = 32
test_dataset = tf_dataset(test_x, test_y, batch=batch_size)
 
test_steps = (len(test_x)//batch_size)
if len(test_x) % batch_size != 0:
    test_steps += 1

In [ ]:
model.evaluate(test_dataset, steps=test_steps)

33/33 [==============================] - 2s 61ms/step - loss: 0.0511 - accuracy: 0.9833 - recall: 0.8911 - precision: 0.9103 - iou: 0.6861


[0.05111163482069969,
 0.983327329158783,
 0.891057550907135,
 0.9103431701660156,
 0.6861231327056885]

In [ ]:
def read_image2(path):
    x = gdal.Open(path).ReadAsArray()
    x = np.rollaxis(x, 0, 3)
    x = (x - minx) / (maxx - minx)
    x = x.astype(np.float32)
    return x

def read_mask2(path):
    x = gdal.Open(path).ReadAsArray()
    #x = np.rollaxis(x, 0, 3)
    x = np.expand_dims(x, axis=-1)
    x = x.astype(np.float32)
    return x

In [ ]:
for i, (x, y) in tqdm(enumerate(zip(test_x, test_y)), total=len(test_x)):
    x = read_image2(x)
    y = read_mask2(y)
    y_pred = model.predict(np.expand_dims(x, axis=0))[0] > 0.9
    h, w, _ = x.shape
    white_line = np.ones((h, 10, 3)) * 255.0
    all_images = [
        x * 255.0, white_line,
        mask_parse(y_pred) * 255.0
    ]
    image = np.concatenate(all_images, axis=1)
    cv2.imwrite(f"results/{i}.png", image)

100%|██████████████████████████████████████████████████████████████████████████████| 1025/1025 [02:27<00:00,  6.94it/s]


In [ ]:
import os, os.path
imgs = []
path2 = "E:\\Citra\\UNet 128\\unet2122\\test_semarang"
valid_images = [".tif"]
for f in os.listdir(path2):
    ext = os.path.splitext(f)[1]
    if ext.lower() not in valid_images:
        continue
    imgs.append(os.path.join(path2,f))
imgs

['E:\\Citra\\UNet 128\\unet2122\\test_semarang\\1_s1_training__1000_128.tif',
 'E:\\Citra\\UNet 128\\unet2122\\test_semarang\\1_s1_training__1001_128.tif',
 'E:\\Citra\\UNet 128\\unet2122\\test_semarang\\1_s1_training__1002_128.tif',
 'E:\\Citra\\UNet 128\\unet2122\\test_semarang\\1_s1_training__1003_128.tif',
 'E:\\Citra\\UNet 128\\unet2122\\test_semarang\\1_s1_training__1004_128.tif',
 'E:\\Citra\\UNet 128\\unet2122\\test_semarang\\1_s1_training__1005_128.tif',
 'E:\\Citra\\UNet 128\\unet2122\\test_semarang\\1_s1_training__1006_128.tif',
 'E:\\Citra\\UNet 128\\unet2122\\test_semarang\\1_s1_training__1007_128.tif',
 'E:\\Citra\\UNet 128\\unet2122\\test_semarang\\1_s1_training__1008_128.tif',
 'E:\\Citra\\UNet 128\\unet2122\\test_semarang\\1_s1_training__1009_128.tif',
 'E:\\Citra\\UNet 128\\unet2122\\test_semarang\\1_s1_training__100_128.tif',
 'E:\\Citra\\UNet 128\\unet2122\\test_semarang\\1_s1_training__1010_128.tif',
 'E:\\Citra\\UNet 128\\unet2122\\test_semarang\\1_s1_training__10

In [ ]:
def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

model2 = keras.models.load_model("unet_flood2122.h5", custom_objects = {"iou": iou})                   

In [ ]:
minx = -30
maxx = 50

for k in tqdm(imgs):
    test = read_image2(k)
    if test.shape==(128,128,3):
        y_pred = model2.predict(np.expand_dims(test, axis=0))[0] > 0.9
        h, w, _ = test.shape
        white_line = np.ones((h, 10, 3)) * 255.0
        all_images = y_pred * 255.0
        #all_images = [
        #mask_parse(y_pred) * 255.0
        #]
        image = np.concatenate(all_images, axis=1)
        cv2.imwrite(os.path.join(path2, f"{k}_2a.png"), image)

100%|██████████████████████████████████████████████████████████████████████████████| 1509/1509 [02:29<00:00, 10.07it/s]


In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(test_y, model.predict(test_x)))

ValueError: in user code:

    C:\Users\Chibonk\anaconda3\envs\unet\lib\site-packages\tensorflow\python\keras\engine\training.py:1462 predict_function  *
        return step_function(self, iterator)
    C:\Users\Chibonk\anaconda3\envs\unet\lib\site-packages\tensorflow\python\keras\engine\training.py:1452 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\Chibonk\anaconda3\envs\unet\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\Chibonk\anaconda3\envs\unet\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\Chibonk\anaconda3\envs\unet\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\Chibonk\anaconda3\envs\unet\lib